# Segment & Cluster Toronto Neighborhoods
by: Diardano Raihan (Indonesia)
<hr>

The project will use the following Wikipedia page as one of data sources.
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

Objective:
- Previously, we have succeeded to retrieve the latitude and longitude coordinate in `Pre2_Coordinate_Retrieval.ipynb`notebook file. 

- Now, we will __explore__, __segment__, and __group neighborhoods__ into clusters to find similar neighborhoods in __Toronto City__.